In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

31-Jan-2017 15:42:24 DEBUG gensim.models.doc2vec: Fast version of gensim.models.doc2vec is being used
31-Jan-2017 15:42:24 INFO summa.preprocessing.cleaner: 'pattern' package not found; tag filters are not available for English
31-Jan-2017 15:42:24 INFO root: Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
31-Jan-2017 15:42:24 INFO root: Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/l

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

31-Jan-2017 14:49:17 INFO preprocessing: Creating document list from TXT files ...
31-Jan-2017 14:49:17 DEBUG preprocessing: 17 entries in document list.


['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

31-Jan-2017 14:49:17 INFO preprocessing: Creating document list from CSV files ...
31-Jan-2017 14:49:17 DEBUG preprocessing: 16 entries in document list.


['corpus_csv/Howard_GodsoftheNorth.txt.csv',
 'corpus_csv/Poe_EurekaAProsePoem.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Poe_ThePurloinedLetter.txt.csv',
 'corpus_csv/Howard_ShadowsintheMoonlight.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

31-Jan-2017 14:49:17 INFO preprocessing: Creating document labels ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Document labels available


['corpus_txt/Poe_EurekaAProsePoem.txt',
 'corpus_txt/Howard_TheDevilinIron.txt',
 'corpus_txt/Lovecraft_TheShunnedHouse.txt',
 'corpus_txt/Howard_SchadowsinZamboula.txt',
 'corpus_txt/Doyle_AStudyinScarlet.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[:1]

31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_EurekaAProsePoem.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_TheDevilinIron.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Lovecraft_TheShunnedHouse.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_SchadowsinZamboula.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Doyle_AStudyinScarlet.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_TheCaskofAmontillado.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Poe_TheMasqueoftheRedDeath.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Howard_GodsoftheNorth.txt ...
31-Jan-2017 14:49:17 DEBUG preprocessing: Accessing TXT document corpus_txt/Kipling_TheEndofthePassage.txt ...
31-Jan-2017 14:49:17 DEBUG

[['eureka',
  'a',
  'prose',
  'poem',
  'by',
  'edgar',
  'a',
  'poe',
  'new-york',
  'geo',
  'p',
  'putnam',
  'of',
  'late',
  'firm',
  'of',
  'wiley',
  'putnam',
  'broadway',
  'mdcccxlviii',
  'entered',
  'according',
  'to',
  'act',
  'of',
  'congress',
  'in',
  'the',
  'year',
  'by',
  'edgar',
  'a',
  'poe',
  'in',
  'the',
  'clerk’s',
  'office',
  'of',
  'the',
  'district',
  'court',
  'for',
  'the',
  'southern',
  'district',
  'of',
  'new-york',
  'leavitt',
  'trow',
  'co',
  'prs',
  'ann-street',
  'with',
  'very',
  'profound',
  'respect',
  'this',
  'work',
  'is',
  'dedicated',
  'to',
  'alexander',
  'von',
  'humboldt',
  'preface',
  'to',
  'the',
  'few',
  'who',
  'love',
  'me',
  'and',
  'whom',
  'i',
  'love—to',
  'those',
  'who',
  'feel',
  'rather',
  'than',
  'to',
  'those',
  'who',
  'think—to',
  'the',
  'dreamers',
  'and',
  'those',
  'who',
  'put',
  'faith',
  'in',
  'dreams',
  'as',
  'in',
  'the',
  'o

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow

0
doc_id token_id    
1      18658      2
       8193       7
       8194       2
       8195      17
       8197       5
       1          2
       19116      1
       16394      1
       16395      1
       2          1
       15         3
       16400      1
       8210       3
       19         1
       16404      2
       16405      1
       25         1
       16410      1
       16412      1
       29         1
       4101       1
       6832       2
       16418      2
       35        10
       38         1
       39        14
       8234      13
       8235       1
       8236      13
       8237       1
...              ..
16     16294      0
       23196      0
       16301      0
       24503      0
       16315      0
       16319      0
       10912      0
       8130       0
       8132       0
       24517      0
       24520      0
       24524      0
       6818       0
       24532      0
       24549      0
       16348      0
       8160       0
       24545      0
       24548      0
       2726       0
       24550      0
       24551      0
       13911      0
       11456      0
       8174       0
       16369      0
       8182       0
       16377      0
       8188       0
       8191       0

[52926 rows x 1 columns]

#### Convert to Doc-Topic Matrix

In [11]:
'''import pandas as pd

doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

cols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]
#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]

set(sparse_bow.index.get_level_values("token_id"))

#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'''

'import pandas as pd\n\ndoc2id = {value : key for key, value in doc_ids.items()}\ntype2id = {value : key for key, value in id_types.items()}\n\ncols = [doc2id[key] for key in set(sparse_bow.index.get_level_values("doc_id"))]\n#idx = [type2id[key] for key in set(sparse_bow.index.get_level_values("token_id"))]\n\nset(sparse_bow.index.get_level_values("token_id"))\n\n#doctopic_matrix = pd.DataFrame(columns=cols, index=idx)'

#### Remove Features

In [12]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "en"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [13]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

31-Jan-2017 14:49:17 INFO preprocessing: Removing features ...
31-Jan-2017 14:49:17 DEBUG preprocessing: 672 features removed.


In [14]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

52926
47660


##### Create Feature Remove Lists

In [15]:
stopword_list = pre.find_stopwords(sparse_bow, id_types, 100)
len(stopword_list)

31-Jan-2017 14:49:18 INFO preprocessing: Finding stopwords ...
31-Jan-2017 14:49:18 DEBUG preprocessing: 100 stopwords found.


100

In [16]:
hapax_list = pre.find_hapax(sparse_bow, id_types)
len(hapax_list)

31-Jan-2017 14:49:18 INFO preprocessing: Find hapax legomena ...
31-Jan-2017 14:49:18 DEBUG preprocessing: 19478 hapax legomena found.


19478

##### Remove Features

In [17]:
feature_list = set(stopword_list).union(hapax_list)
clean_term_frequency = pre.remove_features(sparse_bow, id_types, feature_list)

31-Jan-2017 14:49:18 INFO preprocessing: Removing features ...
31-Jan-2017 14:49:18 DEBUG preprocessing: 19542 features removed.


In [18]:
len(clean_term_frequency)

5685

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [19]:
num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'w', encoding = "utf-8") as f:
    pass

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)

In [20]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [21]:
mm = MmCorpus("gb_plain.mm")

31-Jan-2017 14:49:18 INFO gensim.matutils: initializing corpus reader from gb_plain.mm
31-Jan-2017 14:49:18 INFO gensim.matutils: accepted corpus with 16 documents, 24552 features, 310670 non-zero entries


In [22]:
#in case you're only loading the corpus - build dict first

#import pickle

#pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [23]:
#id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [24]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [25]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [26]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

In [27]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id)

31-Jan-2017 14:49:18 INFO gensim.models.ldamodel: using symmetric alpha at 0.01
31-Jan-2017 14:49:18 INFO gensim.models.ldamodel: using symmetric eta at 4.0728220584042684e-05
31-Jan-2017 14:49:18 INFO gensim.models.ldamodel: using serial LDA version on this node
31-Jan-2017 14:49:43 INFO gensim.models.ldamodel: running online LDA training, 100 topics, 1 passes over the supplied corpus of 16 documents, updating model once every 16 documents, evaluating perplexity every 16 documents, iterating 50x with a convergence threshold of 0.001000
31-Jan-2017 14:49:43 WARNING gensim.models.ldamodel: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
31-Jan-2017 14:50:09 DEBUG gensim.models.ldamodel: bound: at document #0
31-Jan-2017 14:50:12 INFO gensim.models.ldamodel: -35.088 per-word bound, 36521290197.7 perplexity estimate based on a held-out corpus of 16 documents with 310670 words
31-Jan-2017 14:50:12 INFO gensim.models.l

In [28]:
topic_nr_x = model.get_topic_terms(10)

[type2id[i[0]] for i in topic_nr_x]

['ample',
 'bruyère',
 'pack',
 'traced',
 'fallacies',
 'forfeit',
 'demonstrate',
 'forty',
 'hypothetical',
 'imperious']

In [29]:
topics = model.show_topics(num_topics = 60)

In [30]:
topics

[(13,
  '0.066*"ample" + 0.033*"pack" + 0.030*"bruyère" + 0.023*"traced" + 0.021*"fallacies" + 0.018*"imperious" + 0.014*"demonstrate" + 0.011*"forfeit" + 0.010*"hypothetical" + 0.010*"animation"'),
 (37,
  '0.041*"ample" + 0.021*"pack" + 0.020*"bruyère" + 0.016*"imperious" + 0.014*"traced" + 0.013*"fallacies" + 0.009*"forfeit" + 0.009*"step—we" + 0.009*"forty" + 0.008*"demonstrate"'),
 (58,
  '0.045*"ample" + 0.028*"pack" + 0.028*"bruyère" + 0.024*"traced" + 0.020*"fallacies" + 0.017*"imperious" + 0.013*"forty" + 0.012*"dishonour" + 0.011*"step—we" + 0.010*"artillery"'),
 (22,
  '0.053*"ample" + 0.030*"pack" + 0.026*"bruyère" + 0.024*"traced" + 0.019*"demonstrate" + 0.019*"fallacies" + 0.017*"imperious" + 0.015*"forfeit" + 0.014*"step—we" + 0.014*"forty"'),
 (3,
  '0.058*"ample" + 0.034*"bruyère" + 0.029*"pack" + 0.023*"traced" + 0.019*"imperious" + 0.013*"fallacies" + 0.013*"forfeit" + 0.011*"demonstrate" + 0.011*"forty" + 0.010*"step—we"'),
 (95,
  '0.043*"ample" + 0.032*"bruyère" +

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [2]:
import os

#Variable zum Testen -> wenn Mallet unter Linux nicht installiert wurde
path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"

path_to_corpus = os.path.join(os.path.abspath('.'), 'corpus_txt')
assert os.path.exists(path_to_corpus)

#malletBinary = mal.create_mallet_binary(path_to_corpus)
malletBinary = mal.create_mallet_binary(path_to_corpus, path_to_mallet)

31-Jan-2017 15:42:36 DEBUG mallet: tutorial_supplementals/mallet_output/malletBinary.mallet
31-Jan-2017 15:42:36 INFO mallet: Accessing Mallet ...
31-Jan-2017 15:42:37 DEBUG mallet: Mallet file available.


##### Mallet output erzeugen

In [3]:
outfolder = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
mal.create_mallet_model(malletBinary, outfolder, path_to_mallet)

31-Jan-2017 15:42:40 DEBUG mallet: /home/sina/Uni/Dariah/DARIAH-Topics/Topics/tutorial_supplementals/mallet_output
31-Jan-2017 15:42:40 INFO mallet: Accessing Mallet ...
31-Jan-2017 15:42:40 DEBUG mallet: Mallet file available.


##### Matrix von Mallet output erzeugen

In [4]:
basepath = os.path.join(os.path.abspath('.'), "tutorial_supplementals/mallet_output")
doc_topics = os.path.join(basepath, "doc_topics.txt")

In [5]:
mal.create_MalletMatrix(doc_topics)

[[  1.84700883e-02   1.22589081e-02   6.97940503e-02   8.33605754e-03
    2.33736515e-02   3.80353057e-01   7.02844067e-03   8.87544949e-02
    3.75939850e-03   5.83524027e-02   8.33605754e-03   3.77574371e-02
    9.97057862e-03   8.17260543e-04   4.74011115e-03   1.78162798e-02
    1.45635829e-01   9.26773455e-02   3.43249428e-03   8.33605754e-03]
 [  2.27659172e-02   2.05617482e-02   9.19138485e-02   1.10523333e-02
    1.52087663e-02   1.58290824e-01   4.12494490e-03   9.82744505e-02
    1.99319856e-02   7.82480005e-02   6.14018515e-03   2.69223503e-02
    6.26613767e-03   1.98375213e-03   1.66887084e-03   2.09301593e-01
    1.42924617e-01   7.52251401e-02   4.75470748e-03   4.43982619e-03]
 [  7.41623304e-03   1.90876818e-02   1.06866702e-01   2.84491563e-03
    2.79409619e-01   1.15717551e-01   3.33122599e-03   8.66361912e-02
    1.67047610e-02   5.32266693e-02   7.51349511e-03   2.39021544e-02
    5.76277780e-03   5.59256918e-04   1.09419832e-03   1.33978505e-02
    1.70621991e-01

## Visualisierung 

In [ ]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

#### Heatmap

In [ ]:
#heatmap = visual.make_heatmap()

In [ ]:
#visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [ ]:
#vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [ ]:
#vis.make_interactive()

In [ ]:
#vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)